In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
import matplotlib.pyplot as plt
from torch import nn

%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
#%load_ext tensorboard
#%tensorboard --logdir lightning_logs --port 6004

In [5]:
# load the training and test datasets
transform_train = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(size=32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5)) #Normalize to (-1,1)
])

transform_eval = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5)) #Normalize to (-1,1)
])

trainset = torchvision.datasets.MNIST(root='data', train=True,download=True, transform=transform_train)
testset = torchvision.datasets.MNIST(root='data', train=False,download=True, transform=transform_eval)



In [6]:
#Define Dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [7]:
# We can use the exact same way to iterate over samples
for i, item in enumerate(trainloader):
    print('Batch {}'.format(i))
    image, label = item
    print(f"Datatype of Image: {type(image)}")
    print(f"Shape of the Image: {image.shape}")
    print(f"Label Values: {label}")

    if i+1 >= 1:
        break

Batch 0
Datatype of Image: <class 'torch.Tensor'>
Shape of the Image: torch.Size([64, 1, 28, 28])
Label Values: tensor([8, 1, 8, 3, 3, 3, 4, 1, 3, 2, 7, 8, 5, 6, 4, 1, 7, 9, 3, 3, 7, 1, 7, 0,
        9, 9, 8, 2, 0, 7, 1, 7, 1, 3, 3, 7, 3, 7, 7, 0, 6, 9, 8, 0, 3, 0, 8, 7,
        3, 4, 0, 4, 7, 1, 0, 8, 7, 4, 9, 4, 8, 8, 6, 0])


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    print(npimg.shape)
    print(np.transpose(npimg, (1, 2, 0)).shape)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))

(3, 242, 242)
(242, 242, 3)


In [ ]:
class CNNAutoencoder(nn.Module):
    def __init__(self):
        super(CNNAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [1]:
model = CNNAutoencoder()
#model = model.to(device)

NameError: name 'CNNAutoencoder' is not defined

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_uniform_(m.weight,nonlinearity='relu')
        m.bias.data.fill_(0.01)
        
model.apply(init_weights)

In [ ]:
import torch.optim as optim

# Loss function
criterion = nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.Adam(model.parameters(),  lr=0.001, betas=(0.9, 0.999), eps=1e-08, amsgrad=False)
optimizer = torch.optim.Adam(model.parameters(),
                                 lr=0.001, 
                                 weight_decay=1e-5)

In [ ]:
#Initialising the list for storing the loss and accuracy

training_loss = 0.0

for epoch in range(20):
    #Iterating through the minibatches of the data
    for i, data in enumerate(trainloader):
        
        # data is a tuple of (inputs, labels)
        image,_ = data
        #image = image.to(device)
        #y = y.to(device)
        
        # Reset the parameter gradients  for the current  minibatch iteration 
        optimizer.zero_grad()
    
        reconstruction = model(image)             # Perform a forward pass on the network with inputs
        loss = criterion(image, reconstruction) 
        loss.backward()             # Perform a backward pass to calculate the gradients
        optimizer.step()            # Optimise the network parameters with calculated gradients
        training_loss += loss.item()
    
    training_loss = training_loss/len(trainloader)
    print('Epoch: {}'.format(epoch),'\tTraining Loss: {:.4f}'.format(training_loss))

In [ ]:
# Lets get batch of test images
dataiter = iter(testloader)
images, labels = dataiter.next()
output = model(images)                     # get sample outputs
images = images.numpy()                    # prep images for display
output = output.view(64, 1, 28, 28)# resizing output
output = output.detach().numpy()           # use detach when it's an output that requires_grad

# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(25,4))
# input images on top row, reconstructions on bottom
for images, row in zip([images, output], axes):
    for img, ax in zip(images, row):
        ax.imshow(np.squeeze(img), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)